In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#<font color = 'DAF7A6'>Functions Thats Really Needed 

In [2]:
!pip install imbalanced-learn

from imblearn.over_sampling import SMOTE

def balance(X_temp, y_temp):
  smote = SMOTE()
  X_temp, y_temp= smote.fit_resample(X_temp, y_temp)

  return pd.concat([pd.DataFrame(X_temp), pd.DataFrame(y_temp)], axis=1)


from joblib import Parallel, delayed
from google.colab import files
import joblib
  

def save_model(model,model_name):
 joblib.dump(model, model_name)
  
def load_model(model_name):
  return joblib.load(model_name)



def Search_Null(dataset):
  dic={}
  for col in list(dataset.columns):
    rows = []
    flage = 0
    for row in range(dataset.shape[0]):
      if str(dataset[col][row]) == "nan":
        rows.append(row)
        flage = 1
    if flage ==1:
      dic[col] = rows
  
  return dic




def divide_dataset(dataset,percentage,random_state = 0):

  rows,cols = dataset.shape

  if random_state == 0:
    return dataset.iloc[:int(rows*(percentage)),:]
  else:
    import random
    l_r = random.sample(range(0, rows), int(rows*(percentage)))
    return dataset.iloc[l_r,:]


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import numpy as np
import pandas as pd
from tqdm import tqdm

dataset=pd.read_excel("/content/drive/MyDrive/Iqram Sir/Refactored_Dataset.xlsx")



In [4]:

target="Sleep Stage"

classes = list(set(dataset[target]))

In [5]:
dataset.head()

,Sleep Stage,MeanP_Alpha_F4,MedianF_Alpha_F4,MeanF_Alpha_F4,Spectral Edge_Alpha_F4,PeakF_Alpha_F4,MeanP_Beta_F4,MedianF_Beta_F4,MeanF_Beta_F4,Spectral Edge_Beta_F4,...,DAR_G,DTR_F4,DTR_C4,DTR_O2,DTR_G,DTABR_F4,DTABR_C4,DTABR_O2,DTABR_G,P_G
0,W,0.00051,8.74146,18.48195,17.48293,8.24195,0.00071,19.73073,29.84585,33.84195,...,18.191758,8.851240,5.786885,18.564103,11.067409,9.770492,3.833333,8.670455,7.424760,0.009177
1,W,0.00040,10.24000,17.48293,16.85854,10.61463,0.00051,17.23317,26.34927,29.34634,...,16.621053,8.693548,5.173913,28.920000,14.262487,6.604396,3.155556,12.262295,7.340749,0.006355
2,W,0.00036,9.74049,17.98244,17.48293,8.74146,0.00047,17.60780,25.47512,28.22244,...,15.327441,16.389831,7.090909,9.583333,11.021358,12.361446,4.188235,4.618182,7.055954,0.006283
3,W,0.00035,10.11512,17.60780,17.10829,9.61561,0.00047,17.35805,26.47415,28.72195,...,4.718095,5.302326,3.600000,3.640000,4.180775,3.304878,2.067416,1.901639,2.424644,0.002777
4,W,0.00033,9.74049,17.60780,17.23317,8.11707,0.00043,17.23317,25.84976,28.09756,...,8.542423,8.226415,3.921053,7.291667,6.479711,6.434211,2.527027,3.491228,4.150822,0.003691


In [ ]:
Search_Null(dataset)

{}

In [ ]:
dataset.shape

(108450, 112)

In [ ]:
#new_dataset = divide_dataset(dataset,0.1,1)

In [ ]:
set(list(dataset[target]))

{'N1', 'N2', 'N3', 'R', 'W'}

In [6]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
dataset[target]=encoder.fit_transform(dataset[target])

In [ ]:
set(list(dataset[target])) # alphabet wise

{0, 1, 2, 3, 4}

In [8]:
#X = new_dataset.loc[:, new_dataset.columns!=target]
#y = new_dataset[target]
#X = dataset[list_of_feat]
X = dataset.loc[:, dataset.columns!=target]
y = dataset[target]

In [15]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

#apply SelectKBest class to extract top 10 best features
bestfeatures = SelectKBest(score_func=chi2, k=10)
fit = bestfeatures.fit(X,y)

dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)


#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
     
featureScores.nlargest(50,'Score').to_csv("Univarient.csv")


feat = pd.read_csv("/content/Univarient.csv")
list_of_feat = list(feat.loc[:,"Specs"])

In [16]:
len(list_of_feat)

50

In [18]:
X = dataset[list_of_feat]
y = dataset[target]

In [20]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)


In [21]:
new_dataset_2=balance(X_train,y_train)

X_train = new_dataset_2.loc[:, new_dataset_2.columns!=target]
y_train = new_dataset_2[target]

In [22]:
new_dataset_2.shape

(157790, 51)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tqdm import tqdm
N=1000
k_range = range (51,N+1,10)
scores={}
scores_list = []
for k in tqdm(k_range):
  classifier = RandomForestClassifier(n_estimators=k,random_state=0)
  classifier.fit(X_train, y_train)
  y_pred=classifier.predict(X_test)
  scores[k] = accuracy_score(y_test,y_pred)
  scores_list.append(accuracy_score(y_test,y_pred))
  print(str(k)+"/"+str(N)+" round completed......................... Accurecy: "+str(accuracy_score(y_test,y_pred)))



%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize = (25,10))
plt.plot(k_range,scores_list)
plt.xlabel('Value of n_estimators')
plt.ylabel ('Testing Accuracy')



print("The best n_estimators:")
best_estimator=list(scores.keys())[scores_list.index(max(scores_list))]
print(best_estimator)

  1%|          | 1/100 [00:01<02:26,  1.48s/it]

1/1000 round completed......................... Accurecy: 0.5843706777316736


  2%|▏         | 2/100 [00:17<16:09,  9.89s/it]

11/1000 round completed......................... Accurecy: 0.7082065467957585


  3%|▎         | 3/100 [00:47<30:51, 19.09s/it]

...

541/1000 round completed......................... Accurecy: 0.7473951129552789


 56%|█████▌    | 56/100 [6:14:08<9:30:06, 777.43s/it]

551/1000 round completed......................... Accurecy: 0.74716459197787


 57%|█████▋    | 57/100 [6:27:51<9:26:51, 790.97s/it]

561/1000 round completed......................... Accurecy: 0.7469801751959428


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tqdm import tqdm
N=10000
k_range = range (1,N+1)
scores={}
scores_list = []
for k in tqdm(k_range):
  classifier = RandomForestClassifier(max_depth=k,random_state=0)
  classifier.fit(X_train, y_train)
  y_pred=classifier.predict(X_test)
  scores[k] = accuracy_score(y_test,y_pred)
  scores_list.append(accuracy_score(y_test,y_pred))
  print(str(k)+"/"+str(N)+" round completed......................... Accurecy: "+str(accuracy_score(y_test,y_pred)))



%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize = (25,10))
plt.plot(k_range,scores_list)
plt.xlabel('Value of n_estimators')
plt.ylabel ('Testing Accuracy')



print("The best Depth:")
best_depth=list(scores.keys())[scores_list.index(max(scores_list))]
print(best_depth)

  0%|          | 1/10000 [00:21<60:37:00, 21.82s/it]

1/10000 round completed......................... Accurecy: 0.43992623328722913


  0%|          | 2/10000 [00:51<72:48:01, 26.21s/it]

2/10000 round completed......................... Accurecy: 0.5236514522821577


  0%|          | 3/10000 [01:33<92:49:37, 33.43s/it]

...

49/10000 round completed......................... Accurecy: 0.7721069617335178


  0%|          | 50/10000 [2:23:16<587:05:55, 212.42s/it]

50/10000 round completed......................... Accurecy: 0.7719686491470724


  1%|          | 51/10000 [2:26:48<586:29:45, 212.22s/it]

51/10000 round completed......................... Accurecy: 0.7715998155832181


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_estimator = RandomForestClassifier(n_estimators=best_estimator,random_state=0)
rf_estimator.fit(X_train, y_train)
y_pred=rf_estimator.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))

save_model(rf_estimator,"Best_est_"+str(rf_estimator))
#files.download("max_depth_"+str(rf_estimator))

[[ 69 131  10  73 101]
 [  6 302  32  30  14]
 [  0 118 262   2   2]
 [ 18 121   6 223  16]
 [ 32  24   5  14 309]]
              precision    recall  f1-score   support

           0       0.55      0.18      0.27       384
           1       0.43      0.79      0.56       384
           2       0.83      0.68      0.75       384
           3       0.65      0.58      0.61       384
           4       0.70      0.80      0.75       384

    accuracy                           0.61      1920
   macro avg       0.63      0.61      0.59      1920
weighted avg       0.63      0.61      0.59      1920

Accurecy:  0.6067708333333334


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_depth = RandomForestClassifier(max_depth=best_depth,random_state=0)
rf_depth.fit(X_train, y_train)
y_pred=rf_depth.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))

save_model(rf_depth,"max_depth_"+str(rf_depth))
#files.download("max_depth_"+str(rf_depth)) 

[[ 75 134   7  75  93]
 [ 13 291  36  33  11]
 [  0 117 261   4   2]
 [ 21 114   5 231  13]
 [ 35  23   4  15 307]]
              precision    recall  f1-score   support

           0       0.52      0.20      0.28       384
           1       0.43      0.76      0.55       384
           2       0.83      0.68      0.75       384
           3       0.65      0.60      0.62       384
           4       0.72      0.80      0.76       384

    accuracy                           0.61      1920
   macro avg       0.63      0.61      0.59      1920
weighted avg       0.63      0.61      0.59      1920

Accurecy:  0.6067708333333334


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_all = RandomForestClassifier(n_estimators=450,max_depth=39,random_state=0)
rf_all.fit(X_train, y_train)
y_pred=rf_all.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))

save_model(rf_all,"All_"+str(rf_all))
#files.download("max_depth_"+str(rf_all))

[[ 63 134  10  74 103]
 [  8 300  30  33  13]
 [  0 120 260   2   2]
 [ 18 128   6 218  14]
 [ 33  23   4  14 310]]
              precision    recall  f1-score   support

           0       0.52      0.16      0.25       384
           1       0.43      0.78      0.55       384
           2       0.84      0.68      0.75       384
           3       0.64      0.57      0.60       384
           4       0.70      0.81      0.75       384

    accuracy                           0.60      1920
   macro avg       0.62      0.60      0.58      1920
weighted avg       0.62      0.60      0.58      1920

Accurecy:  0.5994791666666667
